In [70]:
import glob
import matplotlib.pylab as plt
import matplotlib.image as mpimg
import numpy as np
import tensorflow as tf
import cv2
import math
import os
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from collections import defaultdict

%env JOBLIB_TEMP_FOLDER=/tmp
K.set_image_dim_ordering('tf')
np.random.seed(0)

In [71]:
neg_data_files = glob.glob("../input/IDC_regular_ps50_idx5/*/0/*.png")
pos_data_files = glob.glob("../input/IDC_regular_ps50_idx5/*/1/*.png")
all_data_files = glob.glob("../input/IDC_regular_ps50_idx5/*/*/*.png")

def process_data(num):
    Y = []
    X = []
    pos_img = []
    neg_img = []
    counter = 0

    for d in neg_data_files[:num]:
        full_size_image = cv2.imread(d)
        add = cv2.resize(full_size_image, (50, 50), interpolation = cv2.INTER_CUBIC)
        X.append(add)
        Y.append(0)
        if counter < 5:
            neg_img.append(add)
        counter += 1
        
    counter = 0
        
    for d in pos_data_files[:num]:
        full_size_image = cv2.imread(d)
        add = cv2.resize(full_size_image, (50, 50), interpolation = cv2.INTER_CUBIC)
        X.append(add)
        Y.append(1)
        if counter < 5:
            pos_img.append(add)
        counter += 1
    
    X = np.array(X).astype(np.float64)
    Y = np.array(Y)
    Y = to_categorical(Y)
    
    return X, Y, neg_img, pos_img

X, Y, imgs0, imgs1 = process_data(15000)

Xtr, Xtest, Ytr, Ytest = train_test_split(X, Y, test_size = 0.2)

print(Xtr.shape)
print(Xtest.shape)

In [72]:
# Drawing images helper functions

def select_random_indices(idc_type, x, y):
    indices = []
    
    while len(indices) < 5:
        index = random.randint(1, x.shape[0])
        if idc_type == 0:
            if np.argmax(y[index]) == 0:
                indices.append(index)
        elif idc_type == 1:
            if np.argmax(y[index]) == 1:
                indices.append(index)
                
    return indices

def get_images(ind_array, x):
    images = []
    for i in ind_array:
        images.append(x[i].astype(np.uint8))
    return images

def show_images(neg, pos):
    for row in range(2):
        plt.figure(figsize=(20, 10))
        for col in range(5):
            if row == 0:
                plt.subplot(1,7,col+1)
                plt.imshow(neg[col])
                plt.axis('off')
                plt.title("IDC(-)")
            else:
                plt.subplot(1,7,col+1)
                plt.imshow(pos[col])
                plt.axis('off')
                plt.title("IDC(+)")

In [73]:
print("Images before pre-processing")

# Get random indices of negative and positive samples
neg_ind = select_random_indices(0, Xtr, Ytr)
pos_ind = select_random_indices(1, Xtr, Ytr)

# Get negative and positive image patches
neg_imgs = get_images(neg_ind, Xtr)
pos_imgs = get_images(pos_ind, Xtr)

show_images(neg_imgs, pos_imgs)

In [74]:
print("Images after subtracting mean")

# Normalize data by subtracting mean
mean_image = np.mean(Xtr)
Xtr -= mean_image
Xtest -= mean_image

# Keep mean_image value for other jupyter notebook
with open('mean_image.txt', 'w') as file:
    file.write(str(mean_image))

# Get same images after normalization
norm_neg_imgs = get_images(neg_ind, Xtr)
norm_pos_imgs = get_images(pos_ind, Xtr)

show_images(norm_neg_imgs, norm_pos_imgs)

In [75]:
def create_model(learn_rate):

    model = Sequential()

    model.add(Convolution2D(32, kernel_size=(3, 3), activation='relu', input_shape = (50, 50, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.50))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    opt = Adam(lr=learn_rate)

    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model

In [76]:
best_acc = float('-inf')
best_params = None
best_model = None

def find_hyperparams():
    learn_rate = [5e-4, 5e-6, 1e-2]
    epochs = [5, 10, 20, 40]
    batch_size = [32, 64, 128]

    for b in batch_size:
        for e in epochs:
            for lr in learn_rate:
                model = create_model(lr)

                model.fit(Xtr, Ytr, validation_data = (Xtest, Ytest),
                              batch_size = b,
                              epochs = e,
                              shuffle = True)

                scores = model.evaluate(Xtest, Ytest, verbose=0)

                print('With parameters batch size = ' + str(b) + ", epochs = " + str(e) + ", learn_rate = " + str(lr))
                print('Test loss:', scores[0])
                print('Test accuracy:', scores[1])

                if scores[1] > best_acc:
                    print("New best model")
                    best_acc = scores[1]
                    best_model = model
                    best_params = [b, e, lr]

    print("Best parameters: ", best_params)
    best_model.save('best_model.h5')
    
    
best_model = create_model(5e-4)
history = best_model.fit(Xtr, Ytr, validation_data = (Xtest, Ytest),
              batch_size = 64,
              epochs = 20,
              shuffle = True)

best_model.save('best_model.h5')

In [77]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

scores = best_model.evaluate(Xtest, Ytest, verbose=0)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

with open('test_accuracy.txt', 'w') as file:
    file.write(str(scores[1]))

Y_pred = best_model.predict(Xtest)

def create_cm(y_pred, y_actual):
    err = defaultdict(int)
    
    for p in range(len(Y_pred)):
        if (Y_pred[p][0] < Y_pred[p][1]) and np.argmax(y_actual[p]) == 0:
            err["false_pos"] += 1
            
        elif (Y_pred[p][0] > Y_pred[p][1]) and np.argmax(y_actual[p]) == 1:
            err["false_neg"] += 1

        elif (Y_pred[p][0] < Y_pred[p][1]) and np.argmax(y_actual[p]) == 1:
            err["true_pos"] += 1
            
        elif (Y_pred[p][0] > Y_pred[p][1]) and np.argmax(y_actual[p]) == 0:
            err["true_neg"] += 1
          
    cm = [[err["true_neg"], err["false_pos"]], [err["false_neg"], err["true_pos"]]]
    return cm

def show_conf_matrix(cm):
    plt.clf()
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Pastel1)
    classNames = ['IDC (-)','IDC (+)']
    plt.title('IDC (-) and IDC (+) Confusion Matrix')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    tick_marks = np.arange(len(classNames))
    plt.xticks(tick_marks, classNames, rotation=45)
    plt.yticks(tick_marks, classNames)
    s = [['TN','FP'], ['FN', 'TP']]
    for i in range(2):
        for j in range(2):
            plt.text(j, i, str(s[i][j]) + " = " + str(cm[i][j]), horizontalalignment="center")
    plt.show()
    
conf_matrix = create_cm(Y_pred, Ytest)
show_conf_matrix(conf_matrix)

In [78]:
def score_IDC(model, patient_no):
    data_files = glob.glob("../input/IDC_regular_ps50_idx5/" + str(patient_no) + "/*/*.png")
    X = []
    Y = []

    for d in data_files:
        full_size_image = cv2.imread(d)
        X.append(cv2.resize(full_size_image, (50, 50), interpolation = cv2.INTER_CUBIC))
        if d.endswith("class0.png"):
            Y.append(0)
        else:
            Y.append(1)
            
    X = np.array(X, dtype=np.float64)
    X -= mean_image
    
    Y_pred = model.predict(X)
    pos = 0.0
    err = defaultdict(int)
    for p in range(len(Y_pred)):
        if Y_pred[p][0] < Y_pred[p][1]:
            pos += 1.0
        if (Y_pred[p][0] < Y_pred[p][1]) and Y[p] == 0:
            err["false_pos"] += 1
        elif (Y_pred[p][0] > Y_pred[p][1]) and Y[p] == 1:
            err["false_neg"] += 1
            
    score = pos / float(len(Y_pred))
    print("Patient: {} --> IDC aggressiveness score: {:0.4F}".format(patient_no, score))
    return score, err

score, err = score_IDC(best_model, 16896)

print("False positives: " + str(err["false_pos"]))
print("False negatives: " + str(err["false_neg"]))
